In [1]:
import sqlite3 as sql3
import pandas as pd
from pandas import DataFrame

In [2]:
conn = sql3.connect('data/prtr_en.db')
data = pd.read_sql_query("""
SELECT f.name AS LPS, a.business_sector AS GNFR, a.prtr_key AS PRTR,  
f.administrative_number AS 'E-PRTR/ PRTR Facility ID', f.wgs84_x AS 'Longitude (deg)', f.wgs84_y AS 'Latitude (deg)',

SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Nitrogen oxides (NOx/NO2)') AS "NOx/NO2 (kt)",
SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Non-methane volatile organic compounds (NMVOC)') AS "(NMVOC) (kt)",
SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Sulphur oxides (SOx/SO2)') AS "SOx/SO2 (kt)",
SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Ammonia (NH3)') AS "NH3 (kt)",
SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Particulate matter (PM2.5)') AS "PM2.5 (kt)",
SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Particulate matter (PM10)') AS "PM10 (kt",
SUM(r.annual_load/1000000) FILTER (WHERE substance_name = 'Carbon monoxide (CO)') AS "CO (kt)",
SUM(r.annual_load/1000) FILTER (WHERE substance_name = 'Lead and compounds (as Pb)') AS "Pb (t)",
SUM(r.annual_load/1000) FILTER (WHERE substance_name = 'Cadmium and compounds (as Cd)') AS "Cd (t)",
SUM(r.annual_load/1000) FILTER (WHERE substance_name = 'Mercury and compounds (as Hg)') AS "Hg (t)",
SUM(r.annual_load*1000) FILTER (WHERE substance_name = 'PCDD + PCDF (dioxins + furans)(as Teq)') 
AS "PCDD/ PCDF (dioxins/ furans)(g I-Teq)",
SUM(r.annual_load/1000) FILTER (WHERE substance_name = 'Polycyclic aromatic hydrocarbons (PAHs)') AS "PAHs (t)",
SUM(r.annual_load) FILTER (WHERE substance_name = 'Hexachlorobenzene (HCB)') AS "HCB (kg)",
SUM(r.annual_load) FILTER (WHERE substance_name = 'Polychlorinated biphenyls') AS "PCBs (kg)"

FROM facilities AS f
JOIN releases AS r ON f.id = r.facility_ID
JOIN activities AS a USING (facility_ID)

WHERE f.year = '2015' AND r.compartment = 'Air' AND a.main_activity = 1
GROUP BY f.id
ORDER BY name

""", conn)
conn.close()
data

,LPS,GNFR,PRTR,E-PRTR/ PRTR Facility ID,Longitude (deg),Latitude (deg),NOx/NO2 (kt),(NMVOC) (kt),SOx/SO2 (kt),NH3 (kt),PM2.5 (kt),PM10 (kt,CO (kt),Pb (t),Cd (t),Hg (t),PCDD/ PCDF (dioxins/ furans)(g I-Teq),PAHs (t),HCB (kg),PCBs (kg)
0,"""Randow,Welse"" GmbH & Co.Agrarprod.KG",Intensive livestock production and aquaculture,7.a.ii,12-20732150000,14.148267,53.204550,NaN,NaN,NaN,0.0144,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,1Heiz Energie GmbH Binnenhafen Eberswalde,Energy sector,1.c,12-20603020000,13.778471,52.851264,0.168,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,3M Technical Ceramics Zweigniederlassung der 3...,Chemical industry,4.b.v,06-09-763-0005-0001,10.329009,47.705599,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,A & L Möller GmbH,Intensive livestock production and aquaculture,7.a.ii,03-86-45404287880,7.689882,52.907102,NaN,NaN,NaN,0.0130,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,A + J Agrar GmbH & Co. KG,Intensive livestock production and aquaculture,7.a.iii,10002,11.508175,51.216046,NaN,NaN,NaN,0.0130,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,swb Erzeugung AG & Co. KG / Heizkraftwerk Hafen,Energy sector,1.c,06-04-11/2001178/8/1,8.727338,53.123907,1.070,NaN,0.961,0.0111,None,NaN,NaN,NaN,NaN,0.0166,None,NaN,NaN,None
1484,swb Erzeugung AG & Co. KG / Heizkraftwerk Hastedt,Energy sector,1.c,06-04-11/2001178/8/0,8.873020,53.059266,0.576,NaN,0.272,NaN,None,NaN,NaN,NaN,NaN,0.0120,None,NaN,NaN,None
1485,v. Wendorff KG DaWa,Intensive livestock production and aquaculture,7.a.ii,13-30-396,12.550303,53.963469,NaN,NaN,NaN,0.0149,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1486,van Bebber Jens,Intensive livestock production and aquaculture,7.a.ii,03-88-45602000040,7.248048,52.304503,NaN,NaN,NaN,0.0175,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [3]:
data.fillna("")

,LPS,GNFR,PRTR,E-PRTR/ PRTR Facility ID,Longitude (deg),Latitude (deg),NOx/NO2 (kt),(NMVOC) (kt),SOx/SO2 (kt),NH3 (kt),PM2.5 (kt),PM10 (kt,CO (kt),Pb (t),Cd (t),Hg (t),PCDD/ PCDF (dioxins/ furans)(g I-Teq),PAHs (t),HCB (kg),PCBs (kg)
0,"""Randow,Welse"" GmbH & Co.Agrarprod.KG",Intensive livestock production and aquaculture,7.a.ii,12-20732150000,14.148267,53.204550,,,,0.0144,,,,,,,,,,
1,1Heiz Energie GmbH Binnenhafen Eberswalde,Energy sector,1.c,12-20603020000,13.778471,52.851264,0.168,,,,,,,,,,,,,
2,3M Technical Ceramics Zweigniederlassung der 3...,Chemical industry,4.b.v,06-09-763-0005-0001,10.329009,47.705599,,,,,,,,,,,,,,
3,A & L Möller GmbH,Intensive livestock production and aquaculture,7.a.ii,03-86-45404287880,7.689882,52.907102,,,,0.013,,,,,,,,,,
4,A + J Agrar GmbH & Co. KG,Intensive livestock production and aquaculture,7.a.iii,10002,11.508175,51.216046,,,,0.013,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,swb Erzeugung AG & Co. KG / Heizkraftwerk Hafen,Energy sector,1.c,06-04-11/2001178/8/1,8.727338,53.123907,1.07,,0.961,0.0111,,,,,,0.0166,,,,
1484,swb Erzeugung AG & Co. KG / Heizkraftwerk Hastedt,Energy sector,1.c,06-04-11/2001178/8/0,8.873020,53.059266,0.576,,0.272,,,,,,,0.012,,,,
1485,v. Wendorff KG DaWa,Intensive livestock production and aquaculture,7.a.ii,13-30-396,12.550303,53.963469,,,,0.0149,,,,,,,,,,
1486,van Bebber Jens,Intensive livestock production and aquaculture,7.a.ii,03-88-45602000040,7.248048,52.304503,,,,0.0175,,,,,,,,,,


In [4]:
data.to_csv('output/Convert ePRTR data to CLRTAP LPS.csv', index=False)